In [1]:
Needs["NDSolve`FEM`"]

In [2]:
Subscript[c, air] = QuantityMagnitude[ThermodynamicData["Air", "SoundSpeed"]];
Subscript[\[Rho], air] = QuantityMagnitude[ThermodynamicData["Air", "Density"]];

In [9]:
t = a/2;
\[Theta] = ArcTan[(b - a)/l];
parametersRules = {r -> 1, a -> 1/15, b -> 3/10, l -> 1/2, d -> 1/2};

In [22]:
rec1 = Rectangle[{-r, a}, {-r + d, a + t}];
rec2 = Polygon @@ (RotationTransform[\[Theta], {-r + d, a}][#] & /@ 
    (List @@ RegionBoundary[Rectangle[{-r + d, a}, {-r + d + l/Cos[\[Theta]], a + t}]]));
\[CapitalOmega] = RegionDifference[Disk[{0, 0}, r, {0, \[Pi]}], RegionUnion[rec1, rec2]] /. parametersRules;

In [25]:
\[Lambda] = (c/Subscript[f, max]) /. {c -> Subscript[c, air], Subscript[f, max] -> 1000};
h = \[Lambda]/12;

In [27]:
mesh = ToElementMesh[\[CapitalOmega], MaxCellMeasure -> {"Length" -> h}]

ElementMesh[{{-1., 1.}, {0., 1.}}, {TriangleElement[<13733>]}]

In [28]:
inlet = Circle[{0, 0}, r, {\[Pi] - ArcSin[a/r], \[Pi]}] /. parametersRules;
Subscript[\[CapitalGamma], in] = 
  AcousticRadiationValue[{x, y} \[Element] inlet, vars, 
   pars, <|"SoundIncidentPressure" -> 1|>];

In [30]:
outlet = Circle[{0, 0}, r, {0, \[Pi] - ArcSin[(a + t)/r]}] /. 
  parametersRules; Subscript[\[CapitalGamma], out] = 
 AcousticAbsorbingValue[{x, y} \[Element] outlet, vars, 
   pars, <|"AcousticSourceDistance" -> 1/(2 r)|>] /. parametersRules;

In [32]:
fRange = Table[f, {f, 50, 1000, 10}];
pfun = ParametricNDSolveValue[pde, p, {x, y} \[Element] mesh, {\[Omega]}];
Monitor[pfunTable = Table[pfun[\[Omega]] /. {\[Omega] -> 2 \[Pi] f}, {f, fRange}], Row[{"frequency being solved = ", CForm[f], " Hz"}]];

In [46]:
f = 900;
{index} = Flatten[Position[fRange, f]];

In [48]:
pMax = Max[Abs[pfunTable[[index]]["ValuesOnGrid"]]];
legendBar = BarLegend[{"TemperatureMap", {-pMax, pMax}}, Sequence[50, LegendLabel -> Style["[Pa]", Opacity[0.6]]]];
options = {PlotRange -> {-pMax, pMax}, Sequence[
   ColorFunction -> ColorData[{"TemperatureMap", {-pMax, pMax}}], 
    ContourStyle -> None, ColorFunctionScaling -> False, 
    Contours -> 30, PlotPoints -> 41, FrameLabel -> {"x", "y"}, 
    PlotLabel -> Style["Transient sound pressure: p(x,y)", 18], 
    AspectRatio -> Automatic, ImageSize -> Medium]};

In [59]:
R = 2;
{x0, y0} = {x, y} /. 
   Last[NSolve[{x^2 + (y - b)^2 == R^2, (x + r)^2 + (y - a)^2 == 
       R^2}, {x, y}]];
\[Phi] = VectorAngle[{-r - x0, a - y0}, {-x0, b - y0}];
\[Gamma] = 3/2 \[Pi] - VectorAngle[{0, -1}, {-r - x0, a - y0}];
curve = Annulus[{x0, y0}, {R - t, R}, {\[Gamma], \[Gamma] + \[Phi]}];
\[CapitalOmega]cur = 
  RegionDifference[Disk[{0, 0}, r, {0, \[Pi]}], curve] /. 
   parametersRules;

In [66]:
pfun2 = ParametricNDSolveValue[pde, 
   p, {x, y} \[Element] meshCur, {\[Omega]}];
Monitor[pfunTable2 = 
   Table[pfun2[\[Omega]] /. {\[Omega] -> 2 \[Pi] f}, {f, fRange}], 
  Row[{"frequency being solved = ", CForm[f], " Hz"}]];

In [68]:
boundaryHighlight2 = Graphics[{{Dashed, outlet}, inlet, 
Line[{{-r, 0}, {r, 0}}], 
Circle[{x0, y0}, R, {\[Gamma], \[Gamma] + \[Phi]}], 
Circle[{x0, y0}, R - t, {\[Gamma], \[Gamma] + \[Phi]}], 
Line[{{0, b}, {
       x0 + (R - t) Cos[\[Gamma] + \[Phi]], 
        y0 + (R - t) Sin[\[Gamma] + \[Phi]]}}]}] /. parametersRules;

In [69]:
nframes = 50;
frames2 = Table[
   Show[Legended[
     ContourPlot[
      Re[pfunTable2[[index]][x, y]*
         Exp[I \[Omega] t] /. {\[Omega] -> 2 \[Pi] f}], {x, 
        y} \[Element] \[CapitalOmega]cur, Evaluate[options]], 
     legendBar], boundaryHighlight2], {t, 0, 0.002, 0.002/nframes}];
frames = Rasterize[#1, "Image", ImageResolution -> 80] & /@ frames;

In [73]:
Subscript[L, in] = N@ArcLength[inlet];

In [74]:
Jrec = Table[(Abs[NIntegrate[pfunTable[[i]][x, y], {x, y} \[Element] inlet]/ Subscript[L, in] - 1]), {i, fRange // Length}];
Jcur = Table[(Abs[NIntegrate[pfunTable2[[i]][x, y], {x, y} \[Element] inlet]/ Subscript[L, in] - 1]), {i, fRange // Length}];